### Ingestion de la carpeta "movie_company"


In [0]:
dbutils.widgets.text("p_environment", "")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/common_functions"

In [0]:
%run "../includes/configuration"

#### Paso 1 - Leer carpeta usando "DataFrameReader" de spark

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [0]:
movies_companies_schema = StructType(fields=[
    StructField("movieId", IntegerType(), True),
    StructField("companyId", IntegerType(), True)
])

In [0]:
movies_companies_df = spark.read \
                .schema(movies_companies_schema) \
                .csv(f"{bronze_folder_path}/{v_file_date}/movie_company")

In [0]:
display(movies_companies_df)

#### Paso 2 - Renombrar las columnas y añadir nuevas
1. "movieId" renombrar a "movie_id"
2. "companyId" renombrar a "company_id"
3. agregar las columnas "ingestion_date" y "environment"

In [0]:
movies_companies_finals_df = add_ingestion_date(movies_companies_df) \
                                .withColumnsRenamed({"movieId": "movie_id",
                                              "companyId": "company_id"}) \
                                .withColumn("environment", lit(v_environment)) \
                                .withColumn("file_date", lit(v_file_date))

In [0]:
display(movies_companies_finals_df)

#### Paso 3 - Eliminar columnas no deseadas

In [0]:
#movies_cast_final_df = movies_casts_with_columns_df.drop(col("genderId"), col("castOrder"))

In [0]:
#display(movies_cast_final_df)

#### Paso 4 - Escribir en parquet

In [0]:
#overwrite_partition(movies_companies_finals_df, "movie_silver", "movies_companies", "file_date")

In [0]:
merge_condition = 'tgt.movie_id = src.movie_id AND tgt.company_id = src.company_id AND tgt.file_date = src.file_date'
merge_delte_lake(movies_companies_finals_df, "movie_silver", "movies_companies", silver_folder_path, merge_condition, "file_date")

In [0]:
%sql
SELECT file_date, count(1)
FROM movie_silver.movies_companies
GROUP BY file_date;

In [0]:
dbutils.notebook.exit("success")